# **Tugas Pengantar Text Mining (Rekognisi)**
---

- **Nama :** Felix
- **NIM :** M0721028
- **Pengampu :** Mr. Fajar Muslim S.T., M.T.


---
## **Import Library**

Disini hanya melakukan import library math untuk pembuatan syntax TF-IDF dan BoW secara manual mengikuti rumus matematika dari metode masing-masing dan library sklearn untuk membantu melakukan modeling menggunakan **Decision Tree**.

In [ ]:
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

---
## **Membuat Data**

Tahap ini adalah membuat data contoh dan dipisahkan untuk memastikan mana variabel independen dan variabel dependen. Keterangan Label adalah sebagai berikut :
- **P** artimya kalimat dengan sentimen **Positif**
- **Ne** artimya kalimat dengan sentimen **Netral**
- **N** artimya kalimat dengan sentimen **Negatif**

In [ ]:
texts = [
    "aku suka statistika",
    "kelasnya bersih",
    "AC-nya kurang dingin",
    "papan tulisnya putih bersih",
    "kursinya kurang empuk"
]

labels = ["P", "P", "N", "Ne", "N"]

---
## **Proses Tokenisasi**

Pada proses ini terdapat 3 tahap yang berbeda, yaitu :

1. Membuat fungsi/definisi tokenisasi untuk memisahkan tiap kata dan juga mengaplikasikan Lower Case untuk semua kata.
2. Mengaplikasikan fungsi yang sudah dibuat untuk menyimpan kata yang "dipisahkan" di `tokenized_texts` dan menyimpan kata unik di `vocab_set`.
3. Menjadikan kata unik yang telah disimpan tadi dalam bentuk **list** dan dilakukan `sort` untuk mengurutkan list kata secara alfabetikal.

In [ ]:
def tokenize(text):
    return text.lower().split()

vocab_set = set()
tokenized_texts = []
for t in texts:
    tokens = tokenize(t)
    tokenized_texts.append(tokens)
    for token in tokens:
        vocab_set.add(token)

vocab_list = list(vocab_set)
vocab_list.sort()

# Menampilkan Vocab List
print("=== Vocab List ===")
print(vocab_list)

=== Vocab List ===
['ac-nya', 'aku', 'bersih', 'dingin', 'empuk', 'kelasnya', 'kurang', 'kursinya', 'papan', 'putih', 'statistika', 'suka', 'tulisnya']


---
## **Proses TF-IDF dan BoW (Bag of Words)**

### **TF-IDF**

Pada proses perhitungan **TF-IDF** terdapat 4 tahap, yaitu:

1. Menghitung jumlah kemunculan setiap kata dalam dokumen, lalu membaginya dengan total kata dalam dokumen tersebut.  
2. Menghitung berapa banyak dokumen yang mengandung setiap kata unik dari semua dokumen.  
3. Menghitung bobot berdasarkan seberapa jarang kata muncul di seluruh dokumen, dimana semakin jarang kata tersebut muncul, semakin tinggi bobotnya.
4. Mengalikan hasil **TF** dan **IDF** untuk mendapatkan nilai akhir, nilai ini akan digunakan untuk mewakili seberapa penting sebuah kata dalam dokumen tertentu.


In [ ]:
def TF_IDF(tokenized_docs, vocab):
    N = len(tokenized_docs)  # jumlah dokumen

    # df[t] = berapa dokumen yang mengandung kata t
    df = {}
    for v in vocab:
        df[v] = 0
    for tokens in tokenized_docs:
        unique_tokens = set(tokens)
        for t in unique_tokens:
            if t in df:
                df[t] += 1

    # Matrix TF-IDF
    tf_idf_matrix = []
    for tokens in tokenized_docs:
        row = []
        total_kata = len(tokens)
        for v in vocab:
            # TF
            tf = tokens.count(v) / float(total_kata) if total_kata > 0 else 0
            # IDF
            idf = math.log((N+1) / (df[v]+1)) + 1

            tf_idf = tf * idf # Disini hasil TF dikalikan dengan IDF sesuai dengan rumus matematikanya
            row.append(tf_idf)
        tf_idf_matrix.append(row)
    return tf_idf_matrix

tfidf_matrix = TF_IDF(tokenized_texts, vocab_list)

# Menampilkan hasil TF-IDF
print("\n=== TF-IDF Matrix ===")
for i, row in enumerate(tfidf_matrix):
    print(f"Dokumen {i} ({texts[i]}): {row}")


=== TF-IDF Matrix ===
Dokumen 0 (aku suka statistika): [0.0, 0.6995374295560366, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6995374295560366, 0.6995374295560366, 0.0]
Dokumen 1 (kelasnya bersih): [0.0, 0.0, 0.8465735902799727, 0.0, 0.0, 1.049306144334055, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Dokumen 2 (AC-nya kurang dingin): [0.6995374295560366, 0.0, 0.0, 0.6995374295560366, 0.0, 0.0, 0.5643823935199818, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Dokumen 3 (papan tulisnya putih bersih): [0.0, 0.0, 0.42328679513998635, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5246530721670275, 0.5246530721670275, 0.0, 0.0, 0.5246530721670275]
Dokumen 4 (kursinya kurang empuk): [0.0, 0.0, 0.0, 0.0, 0.6995374295560366, 0.0, 0.5643823935199818, 0.6995374295560366, 0.0, 0.0, 0.0, 0.0, 0.0]


### **Bag of Words**

Pada proses **Bag of Words (BOW)** terdapat 3 tahap, yaitu:

1. Membuat Kosakata dengan mengumpulkan semua kata unik dari seluruh dokumen dan menyimpannya dalam bentuk *list* yang terurut.  
2. Menghitung Frekuensi Kata dimana untuk setiap dokumen, akan dihitung berapa kali setiap kata dalam kosakata muncul. dan jika kata tidak muncul dalam dokumen, nilainya adalah 0.
3. Membangun **Matriks BoW** yang dimana hasil perhitungan frekuensi setiap kata dalam dokumen disusun menjadi matriks. Baris merepresentasikan dokumen dan kolom merepresentasikan kata dari kosakata. dan juga nilai dalam matriks adalah frekuensi kata di dokumen tersebut.


In [ ]:
def BoW(tokenized_docs, vocab):
    bow_matrix = []
    for tokens in tokenized_docs:
        row = []
        for v in vocab:
            count = tokens.count(v)
            row.append(count)
        bow_matrix.append(row)
    return bow_matrix

bow_matrix = BoW(tokenized_texts, vocab_list)

# Menampilkan hasil BoW

print("\n=== BoW Matrix ===")
for i, row in enumerate(bow_matrix):
    print(f"Dokumen {i} ({texts[i]}): {row}")


=== BoW Matrix ===
Dokumen 0 (aku suka statistika): [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]
Dokumen 1 (kelasnya bersih): [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
Dokumen 2 (AC-nya kurang dingin): [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Dokumen 3 (papan tulisnya putih bersih): [0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1]
Dokumen 4 (kursinya kurang empuk): [0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0]


---
## **Modeling**

Mengubah label dari bentuk *string* ke *integer*

In [ ]:
label_map = {"P":0, "N":1, "Ne":2}
y = [label_map[l] for l in labels]

Melakukan split *train-test* untuk persiapan melakukan *modeling* dengan rasio 60:40. dengan 3 kalimat sebagai *data train* dan 2 kalimat sebagai *data test*

In [ ]:
X_bow_train, X_bow_test, y_train, y_test = train_test_split(bow_matrix, y, test_size=0.4, random_state=2024)
X_tfidf_train, X_tfidf_test, y_train, y_test = train_test_split(tfidf_matrix, y, test_size=0.4, random_state=2024)

Melakukan pelatihan menggunakan model **Decision Tree** untuk kedua data.

In [ ]:
model_bow = DecisionTreeClassifier(random_state=2024)
model_bow.fit(X_bow_train, y_train)

model_tfidf = DecisionTreeClassifier(random_state=2024)
model_tfidf.fit(X_tfidf_train, y_train)

DecisionTreeClassifier(random_state=2024)

---
## **Model Evaluation**

Melakukan prediksi pada `y_test`.

In [ ]:
y_pred_bow = model_bow.predict(X_bow_test)
acc_bow = accuracy_score(y_test, y_pred_bow)

y_pred_tfidf = model_tfidf.predict(X_tfidf_test)
acc_tfidf = accuracy_score(y_test, y_pred_tfidf)

Menunjukkan akurasi untuk model **Decision Tree** pada kedua metode.

In [ ]:
print("Akurasi dengan BOW:", acc_bow)
print("Akurasi dengan TF-IDF:", acc_tfidf)

Akurasi dengan BOW: 0.5
Akurasi dengan TF-IDF: 0.5


---
## **Kesimpulan**

<div align='justify'>
&emsp;&emsp;&emsp;&emsp;
Akurasi 0.5 menunjukkan bahwa model Decision Tree belum optimal karena keterbatasan data yang kecil, serta label yang ada 3 kelas berbeda dari 5 data yang menunjukkan kardinalitas yang tinggi dan tidak baik untuk sebuah model machine learning. Model hanya mampu menangkap pola dasar, namun belum cukup untuk prediksi yang akurat. Menambah data agar lebih banyak untuk memperkuat model atau menggunakan cross-validation mungkin dapat meningkatkan performa dari model.
</div>